In [1]:
from functools import partial
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
(X_train, y_train) , (X_valid, y_valid)   = (X_train_full[:-5000], y_train_full[:-5000]), (X_train_full[-5000:], y_train_full[-5000:])

X_mean = X_train.mean(axis = 0, keepdims = True)
X_std  = X_train.std(axis = 0, keepdims = True) + 1e-7

X_train = (X_train - X_mean)/(X_std)
X_valid = (X_valid - X_mean)/(X_std)
X_test = (X_test - X_mean)/(X_std)


X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]



In [3]:
X_train.shape, X_valid.shape

((55000, 28, 28, 1), (5000, 28, 28, 1))

In [4]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size = 3, activation = tf.nn.relu, padding = "SAME")
DefaultMaxPool2D = partial(keras.layers.MaxPool2D)


In [5]:
model = keras.models.Sequential([
    DefaultConv2D(filters = 64, kernel_size = 7, input_shape = [28,28, 1]),
    DefaultMaxPool2D(pool_size=2), 
    DefaultConv2D(filters = 128), 
    DefaultConv2D(filters = 128), 
    DefaultMaxPool2D(pool_size=2), 
    DefaultConv2D(filters = 256), 
    DefaultConv2D(filters = 256), 
    DefaultMaxPool2D(pool_size=2), 
    keras.layers.Flatten(),
    keras.layers.Dense(units = 128, activation=tf.nn.relu),
    keras.layers.Dropout(0.5), 
    keras.layers.Dense(units = 64, activation=tf.nn.relu),
    keras.layers.Dropout(0.5), 
    keras.layers.Dense(units = 10, activation=tf.nn.softmax)
]
)

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        3200      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 256)         295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 256)         5

In [7]:
model.compile(loss = tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics = ["accuracy"])
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs= 10, workers=0)
score = model.evaluate(X_test, y_test)
X_new = X_test[:10]
y_pred = model.predict(X_new)

Epoch 1/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.7475 - accuracy: 0.7382 - val_loss: 0.3738 - val_accuracy: 0.8596
Epoch 2/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4403 - accuracy: 0.8497 - val_loss: 0.3540 - val_accuracy: 0.8752
Epoch 3/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3783 - accuracy: 0.8713 - val_loss: 0.3213 - val_accuracy: 0.8844
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3374 - accuracy: 0.8850 - val_loss: 0.3468 - val_accuracy: 0.8794
Epoch 5/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3180 - accuracy: 0.8914 - val_loss: 0.2903 - val_accuracy: 0.8918
Epoch 6/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2973 - accuracy: 0.8983 - val_loss: 0.2734 - val_accuracy: 0.9022
Epoch 7/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2881 - accuracy: 0.9029 - val_loss: 0.3033 - val_accuracy: